# Init spark/files/base classes

In [1]:
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [2]:
from pyspark.sql import SparkSession
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'

spark_session = SparkSession.builder.getOrCreate()
sc = spark_session._sc
spark_session

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


### Load data/files

In [3]:
!head /mnt/data/programming-languages.csv

name,wikipedia_url
A# .NET,https://en.wikipedia.org/wiki/A_Sharp_(.NET)
A# (Axiom),https://en.wikipedia.org/wiki/A_Sharp_(Axiom)
A-0 System,https://en.wikipedia.org/wiki/A-0_System
A+,https://en.wikipedia.org/wiki/A%2B_(programming_language)
A++,https://en.wikipedia.org/wiki/A%2B%2B
ABAP,https://en.wikipedia.org/wiki/ABAP
ABC,https://en.wikipedia.org/wiki/ABC_(programming_language)
ABC ALGOL,https://en.wikipedia.org/wiki/ABC_ALGOL
ABSET,https://en.wikipedia.org/wiki/ABSET


In [4]:
prog_lang_schema = StructType([
    StructField("name", StringType(), False),
    StructField("wikipedia_url", StringType(), False),
])
df_prog_langs = spark_session.read.csv("/mnt/data/programming-languages.csv", schema=prog_lang_schema)
df_prog_langs.head()

Row(name='name', wikipedia_url='wikipedia_url')

In [5]:
# List of language names
prog_langs_list = [str(row[0]) for row in df_prog_langs.select('name').collect()]
prog_langs_list[:10]

['name',
 'A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET']

### Check XML file and load it via spark

In [6]:
!head -n 3 /mnt/data/posts_sample.xml

���<?xml version="1.0" encoding="utf-8"?>
<posts>
  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="630" ViewCount="42817" Body="&lt;p&gt;I want to use a track-bar to change a form's opacity.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;Cannot implicitly convert type &lt;code&gt;'decimal'&lt;/code&gt; to &lt;code&gt;'double'&lt;/code&gt;&lt;/p&gt;&#xA;&lt;/blockquote&gt;&#xA;&#xA;&lt;p&gt;I tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt; but then the control doesn't work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserId="8" LastEditorUserId="3641067" LastEditorDisplayName="Rich B" LastEditDate="2019-07-19T01:39:54.173" 

In [7]:
df_posts_sample = spark_session.read.format("xml").options(rowTag="row").load('/mnt/data/posts_sample.xml')
df_posts_sample.head()

Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUse

In [38]:
def define_language(row):
    language_tag = None
    for lang in prog_langs_list:
        if '<' + lang.lower() + '>' in row._Tags.lower():
            language_tag = lang
            break
    if language_tag is None:
        return None
    return (row._Id, language_tag, row._CreationDate.year)

def check_date_range(row):
    left_border = datetime(year=2010, month=1, day=1)
    right_border = datetime(year=2020, month=1, day=1)
    created_date = row._CreationDate
    return created_date > left_border and created_date < right_border

In [39]:
top_languages_per_year = df_posts_sample.rdd.filter(
    lambda row: row._Tags is not None and check_date_range(row)
).map(define_language).filter(
    lambda row: row is not None
).keyBy(lambda row: (row[1], row[2])).aggregateByKey(
    0,
    lambda acc, value: acc + 1,
    lambda acc1, acc2: acc1 + acc2,
).map(
    lambda row: (row[0][0], row[0][1], row[1])
).toDF()
top_languages_per_year.show()

+-----------+----+---+
|         _1|  _2| _3|
+-----------+----+---+
| JavaScript|2010| 44|
|       Java|2011| 92|
| ColdFusion|2011|  4|
|      Scala|2013| 10|
|     Python|2013| 87|
|Objective-C|2014| 49|
|       Bash|2014| 13|
|       Perl|2014|  9|
|      Scala|2014|  7|
| JavaScript|2016|272|
|     MATLAB|2016| 15|
|       Curl|2016|  5|
|     Python|2016|141|
|        PHP|2019| 59|
| JavaScript|2019|131|
|     Python|2019|162|
|       Curl|2019|  3|
|      Scala|2019|  6|
|     Elixir|2015|  2|
|     Python|2015|119|
+-----------+----+---+
only showing top 20 rows



In [40]:
top_languages_per_year = top_languages_per_year.select(
    col('_1').alias('ProgrammingLanguage'), 
    col('_2').alias('Year'),
    col('_3').alias('NumberOfPosts')
)
top_languages_per_year.show()

+-------------------+----+-------------+
|ProgrammingLanguage|Year|NumberOfPosts|
+-------------------+----+-------------+
|         JavaScript|2010|           44|
|               Java|2011|           92|
|         ColdFusion|2011|            4|
|              Scala|2013|           10|
|             Python|2013|           87|
|        Objective-C|2014|           49|
|               Bash|2014|           13|
|               Perl|2014|            9|
|              Scala|2014|            7|
|         JavaScript|2016|          272|
|             MATLAB|2016|           15|
|               Curl|2016|            5|
|             Python|2016|          141|
|                PHP|2019|           59|
|         JavaScript|2019|          131|
|             Python|2019|          162|
|               Curl|2019|            3|
|              Scala|2019|            6|
|             Elixir|2015|            2|
|             Python|2015|          119|
+-------------------+----+-------------+
only showing top

In [43]:
top_languages_per_year.write.parquet("top_languages_per_year.parquet")

In [49]:
folder_save_name = "reports_top_10languages_per_year"
os.makedirs(folder_save_name, exist_ok=True)

parquetFile = spark_session.read.parquet("top_languages_per_year.parquet")
parquetFile.createOrReplaceTempView("parquetFile")
for year in range(2010, 2020):
    print(year)
    top_10 = spark_session.sql(
        "SELECT ProgrammingLanguage "
        "FROM parquetFile "
        f"WHERE Year = {year} "
        "ORDER BY NumberOfPosts DESC LIMIT 10 "
    )
    top_10.write.parquet(f"{folder_save_name}/top_10_langueages_year_{year}.parquet")
    top_10.show()
    print('--------------')

2010
+-------------------+
|ProgrammingLanguage|
+-------------------+
|               Java|
|         JavaScript|
|                PHP|
|             Python|
|        Objective-C|
|                  C|
|               Ruby|
|             Delphi|
|        AppleScript|
|               Perl|
+-------------------+

--------------
2011
+-------------------+
|ProgrammingLanguage|
+-------------------+
|                PHP|
|               Java|
|         JavaScript|
|             Python|
|        Objective-C|
|                  C|
|               Ruby|
|               Perl|
|             Delphi|
|               Bash|
+-------------------+

--------------
2012
+-------------------+
|ProgrammingLanguage|
+-------------------+
|                PHP|
|         JavaScript|
|               Java|
|             Python|
|        Objective-C|
|                  C|
|               Ruby|
|               Bash|
|                  R|
|                Lua|
+-------------------+

--------------
2013
+-------